<center><font size="19" ><b>Python &nbsp; vs &nbsp; C++  &nbsp; &nbsp;&nbsp; </b></font></center>

<img width='300' src='imgs/py_vs_cpp.svg' />

![raw](./imgs/lambo.jpg)

### code from previous notebooks

In [4]:
from PIL import Image
import ctypes
from ctypes import *
import numpy as np
import time

from AI_Decomposed import *
CPP_recurse_convolution = cdll.LoadLibrary('./cpp/cmake-build-debug/libconvolution.so').recurse_convolution

In [5]:
image_file = './imgs/lambo.jpg'
[height, width, red,green,blue] = image_meta(image_file)

### C++

In [16]:
import time

recurse_cnt = 10

start = time.time()
# create reference to C++ fucntion
cpp_function_ptr = cdll.LoadLibrary('./cpp/cmake-build-debug/libconvolution.so').recurse_convolution
CPP_convolution = native_recurse_pooling(green, width, recurse_cnt, False, cpp_function_ptr)
print ('seconds: ',time.time() - start)

seconds:  2.6742591857910156


### Python

In [17]:
start = time.time()
PY_convolution = PY_recurse_pooling(green, width, np.mean, recurse_cnt)
print ('seconds: ',time.time() - start)

seconds:  136.86554217338562


### verify pixel counts

In [12]:
display(len(CPP_convolution))
display(len(PY_convolution))

817281

817281

### visual verification

In [15]:
recurse_cnt = 10
display_channel(CPP_convolution, width - (recurse_cnt))
display_channel(PY_convolution, width - (recurse_cnt))

TypeError: too many data entries

### performance at  scale

In [14]:
def calc_ttl_pixels(width, height, recursions):    
    w = width
    h = height
    total_pixels = 0
    for i in range(1,(recursions+1),1):    
        total_pixels = total_pixels + (w*h)
        w = w - 1
        h = h - 1
    return total_pixels

In [92]:
import csv
import time

recursions = [1, 5, 10, 20, 40, 80, 160]

print('timestamp,\trecurse_cnt,\tpxls,\tpxls calced,\tcpp_sec,\tcpp_per_sec,\tpy_sec,\tpy_per_sec')

cpp_function_ptr = cdll.LoadLibrary('./cpp/cmake-build-debug/libconvolution.so').recurse_convolution

with open('./results/py_v_cpp.3.csv', mode='w') as py_v_cpp:
    py_v_cpp_writer = csv.writer(py_v_cpp, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    py_v_cpp_writer.writerow(['time','recursion_count','image_size','pixels_calculated','cpp_seconds',
                             'cpp_pixels_per_second','python_seconds','python_pixels_per_second'])
    for recurse_cnt in recursions:

        # timestamp
        cur = time.time()
        dt = datetime.datetime.fromtimestamp(cur).strftime('%H:%M:%S')
        print(dt,   end='', flush=True)
        print(',\t',   end='', flush=True)
        
        # recurse cnt
        print(recurse_cnt, end='', flush=True)
        print(',\t\t',   end='', flush=True)
        
        # pxls
        ttl = height*width    
        print(ttl, end='', flush=True)
        print(',\t',   end='', flush=True)

        # pxls calced
        ttl_pixels_calced = calc_ttl_pixels(width, height, recurse_cnt)
        print(ttl_pixels_calced, end='', flush=True)        
        print(',\t',   end='', flush=True)    
        if recurse_cnt == 1:
            print('\t',   end='', flush=True)            

        start = time.time()
        cpp_conv = native_recurse_pooling(green, width, recurse_cnt, False, cpp_function_ptr)    
        cpp_secs = time.time() - start   

        # cpp time
        print(round(cpp_secs,2), end='', flush=True)
        print(',\t\t',   end='', flush=True)

        # cpp pxls per sec
        cpp_per_sec = ttl_pixels_calced/cpp_secs
        print(round(cpp_per_sec), end='', flush=True)
        print(',\t',   end='', flush=True)

        start = time.time()
        py_conv = PY_recurse_pooling(green, width, np.mean, recurse_cnt)    
        py_secs = time.time() - start

        # py time
        print(round(py_secs,2), end='', flush=True)
        print(',\t',   end='', flush=True)

        #py pxls per sec
        py_pxls_per_sec = ttl_pixels_calced/py_secs
        print(round(py_pxls_per_sec), end='', flush=True)

        print()    
        py_v_cpp_writer.writerow([cur,recurse_cnt,ttl,ttl_pixels_calced,
                                 cpp_secs,cpp_per_sec,py_secs,py_pxls_per_sec])    

    py_v_cpp.flush()
    py_v_cpp.close()

timestamp,	recurse_cnt,	pxls,	pxls calced,	cpp_sec,	cpp_per_sec,	py_sec,	py_per_sec
20:01:58,	1,		819200,	819200,		0.36,		2257791,	13.65,	60030
20:02:12,	5,		819200,	4076830,	1.34,		3037928,	67.12,	60736
20:03:20,	10,		819200,	8105885,	2.59,		3126285,	133.27,	60822
20:05:36,	20,		819200,	16021670,	4.71,		3398786,	268.78,	59609
20:10:09,	40,		819200,	31290940,	8.65,		3616718,	500.83,	62479
20:18:39,	80,		819200,	59636280,	13.99,		4262079,	954.06,	62508
20:34:47,	160,		819200,	108002160,	25.47,		4240475,	1800.18,	59995


In [93]:
import pandas as pd
py_v_cpp_df = pd.read_csv("./results/py_v_cpp.3.csv")

In [94]:
py_v_cpp_df

,time,recursion_count,image_size,pixels_calculated,cpp_seconds,cpp_pixels_per_second,python_seconds,python_pixels_per_second
0,1.554336e+09,1,819200,819200,0.362833,2.257791e+06,13.646427,60030.366875
1,1.554336e+09,5,819200,4076830,1.341977,3.037928e+06,67.124298,60735.532455
2,1.554336e+09,10,819200,8105885,2.592817,3.126285e+06,133.272698,60821.797267
3,1.554336e+09,20,819200,16021670,4.713938,3.398786e+06,268.779000,59609.084086
4,1.554337e+09,40,819200,31290940,8.651751,3.616718e+06,500.826890,62478.554277
5,1.554337e+09,80,819200,59636280,13.992298,4.262079e+06,954.057458,62508.059153
6,1.554338e+09,160,819200,108002160,25.469354,4.240475e+06,1800.175304,59995.356996


# NEXT - visualize results

http://localhost:8888/notebooks/4_V_Visual_Comparison_Python_VS_C%2B%2B.ipynb

<img src='imgs/bokeh_pandas.svg' width='300' />

<img src='imgs/py_v_cpp.png' width='800' />